In [37]:
#pip install pandas folium geopy
import sys
import os


# Adiciona o diretório principal ao sys.path para que possamos importar conexao
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))

# Importa o DataFrame df_membros de conexao.py
from conexao import df_membros


import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from IPython.display import display

# Função para obter coordenadas a partir do CEP
def get_coordinates(cep):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(f"{cep}, Manaus, Brazil")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Aplicar a função aos CEPs do DataFrame
df_membros[['latitude', 'longitude']] = df_membros['cep'].apply(lambda cep: pd.Series(get_coordinates(cep)))

# Filtrar CEPs com coordenadas válidas
valid_coordinates = df_membros.dropna(subset=['latitude', 'longitude'])

# Filtrar coordenadas que estão dentro dos limites de Manaus
def is_within_manaus(lat, lon):
    lat_min, lat_max = -3.2, -2.8
    lon_min, lon_max = -60.1, -59.8
    return lat_min <= lat <= lat_max and lon_min <= lon <= lon_max

valid_coordinates = valid_coordinates[valid_coordinates.apply(lambda row: is_within_manaus(row['latitude'], row['longitude']), axis=1)]

# Criando um mapa base centrado na localização média de Manaus
m = folium.Map(location=[-3.080689, -59.993497], zoom_start=12)

# Adicionando os pontos de calor
heat_data = valid_coordinates[['latitude', 'longitude']].values.tolist()
HeatMap(heat_data).add_to(m)

# Adicionando marcadores com os nomes das pessoas
for index, row in valid_coordinates.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['nome_completo']
    ).add_to(m)

# Salvando o mapa em um arquivo HTML
m.save('mapa_local_membros.html')

# Para visualizar o mapa no Jupyter Notebook
#display(m)
